In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.2 MB/s eta 0:00:00


In [ ]:
import torch 
import re 
from transformers import AutoTokenizer, ViTFeatureExtractor, VisionEncoderDecoderModel 


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
encoder_checkpoint = "nlpconnect/vit-gpt2-image-captioning"
decoder_checkpoint = "nlpconnect/vit-gpt2-image-captioning"
model_checkpoint = "nlpconnect/vit-gpt2-image-captioning"
feature_extractor = ViTFeatureExtractor.from_pretrained(encoder_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(decoder_checkpoint)
model = VisionEncoderDecoderModel.from_pretrained(model_checkpoint).to(device)


Using cuda device


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
def predict(image,max_length=64, num_beams=4):
  image = feature_extractor(image, return_tensors="pt").pixel_values.to(device)
  clean_text = lambda x: x.replace('<|endoftext|>','').split('\n')[0]
  caption_ids = model.generate(image, max_length = max_length)[0]
  caption_text = clean_text(tokenizer.decode(caption_ids))
  return caption_text 

In [ ]:
from PIL import Image

In [ ]:
import pandas as pd
train = pd.read_json("/content/drive/MyDrive/BTP Hate Speech 2022_23/dataset/faceebook memes dataset/data/train.json",lines=True)
test = pd.read_json("/content/drive/MyDrive/BTP Hate Speech 2022_23/dataset/faceebook memes dataset/data/test.json",lines=True)
val = pd.read_json("/content/drive/MyDrive/BTP Hate Speech 2022_23/dataset/faceebook memes dataset/data/dev.json",lines=True)

In [ ]:
base_path='/content/drive/MyDrive/BTP Hate Speech 2022_23/dataset/faceebook memes dataset/data/'

In [ ]:
train=val

In [ ]:
train.head()

,id,img,label,text
0,8291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...
4,80243,img/80243.png,1,mississippi wind chime


In [ ]:
train.shape

(500, 4)

In [ ]:
capt_Text=[]
for ind in train.index:
  im = Image.open(f"{base_path}{train['img'][ind]}")
  im = im.convert('RGB')
  capt_text=predict(im)
  if ind%100==0:
    print(f"Caption text for  index {ind} is {capt_text}")
  capt_Text.append(capt_text)

Caption text for  index 0 is a woman sitting on a curb eating a piece of food 
Caption text for  index 100 is a woman and a man are in a kitchen 
Caption text for  index 200 is a collage of photos showing a collage of a television set 
Caption text for  index 300 is a person holding a hand up to a picture of a person 
Caption text for  index 400 is a woman sitting in a chair with a laptop 


In [ ]:
train['capt_text']=capt_Text

In [ ]:
train.head()

,id,img,label,text,capt_text
0,8291,img/08291.png,1,white people is this a shooting range,a woman sitting on a curb eating a piece of food
1,46971,img/46971.png,1,bravery at its finest,a man holding a sign with a picture of a woman
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,a woman is sitting at a desk with a laptop
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,a man wearing a costume holding a large piece ...
4,80243,img/80243.png,1,mississippi wind chime,a crowd of people standing around a group of p...


In [ ]:
train.to_csv(f'{base_path}val_caption.csv')

In [ ]:
train.to_json(f'{base_path}val_with_caption.json',orient='records')

In [ ]:
df=pd.read_json(f'{base_path}val_with_caption.json')

In [ ]:
df.head()

,id,img,label,text,capt_text
0,8291,img/08291.png,1,white people is this a shooting range,a woman sitting on a curb eating a piece of food
1,46971,img/46971.png,1,bravery at its finest,a man holding a sign with a picture of a woman
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,a woman is sitting at a desk with a laptop
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,a man wearing a costume holding a large piece ...
4,80243,img/80243.png,1,mississippi wind chime,a crowd of people standing around a group of p...


In [ ]:
df.shape

(500, 5)